In [1]:
import sys, os, imp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

C:\Users\alves\AppData\Local\Temp\ipykernel_6780\3692977961.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import sys, os, imp


In [2]:
# lumapiFile = "/opt/lumerical/v221/api/python/lumapi.py"
lumapiFile = "C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
lumapi = imp.load_source("lumapi", lumapiFile)
emeApi = lumapi.MODE(hide = False)

# Parameters


In [3]:
# units
um = 1e-6
nm = 1e-9

# Materials 
sub_material = 'SiO2 (Glass) - Palik'
core_material = 'Si (Silicon) - Palik'


# Waveguide parameters
wvg_width = 450 * nm
wvg_height = 220 * nm
L = 50*um

# Bandwidth operation
cband = True
# C Band simulation: 1520 - 1575 nm
# L Band simulation: 1555 - 1635 nm
if cband:
    wvlngth_start = 1.520 * um
    wvlngth_stop = 1.575 * um
else:
    wvlngth_start = 1.555 * um
    wvlngth_stop = 1.635 * um

nmbrFrqPnts = 20
filename = "wgd_1" 

# Structure creation

In [ ]:
# Defining a Structure Group
emeApi.addstructuregroup()
dev_layer = "GDS_LAYER 35:4"
emeApi.set("name", dev_layer)

In [5]:
emeApi.switchtolayout()
emeApi.deleteall()

emeApi.addrect()
emeApi.addtogroup(dev_layer)
emeApi.set("name", filename)
emeApi.set("x", 0)
emeApi.set("x span", L)
emeApi.set("y", 0)
emeApi.set("y span", wvg_width)
emeApi.set("z", 0)
emeApi.set("z span", wvg_height)
emeApi.set("material", core_material)

# Create solver

In [6]:
fde_solve_material = sub_material

# Define dimensions and other characteristics
x_fde = 0*nm
z_span_fde = wvg_height*5
y_fde = 0*um
y_span_fde = wvg_width*5
z_fde = 0.0 * um

mesh_cells = 100
mesh_multiplier = 8
number_modes = 4

wavelength = 1550.0*nm

In [7]:
# Add solver
emeApi.switchtolayout()
emeApi.select('FDE')
emeApi.delete()
emeApi.select('mesh')
emeApi.delete()

emeApi.addfde()
emeApi.set("solver type", "2D X normal")
emeApi.set("background material", fde_solve_material)
emeApi.set("x", x_fde)
emeApi.set("y span", y_span_fde)
emeApi.set("y", y_fde)
emeApi.set("z span", z_span_fde )
emeApi.set("z", z_fde)
emeApi.set("z min bc", "Metal")
emeApi.set("z max bc", "Metal")
emeApi.set("y min bc", "Metal")
emeApi.set("y max bc", "Metal")
emeApi.set("mesh cells z", mesh_cells)
emeApi.set("mesh cells y", mesh_cells)
emeApi.set('number of trial modes', number_modes)
emeApi.set('wavelength', wavelength)

# Add mesh
emeApi.addmesh()
emeApi.set('set mesh multiplier', True)
emeApi.set('x', x_fde)
emeApi.set('x span', 0)
emeApi.set('x mesh multiplier', mesh_multiplier)
emeApi.set('y', y_fde)
emeApi.set('y span', wvg_width*1.2)
emeApi.set('y mesh multiplier', mesh_multiplier)
emeApi.set('z span', wvg_height*1.2)
emeApi.set('z', z_fde)

In [8]:
emeApi.findmodes()
n1 = emeApi.getresult('FDE::data::mode1', 'neff')
n2 = emeApi.getresult('FDE::data::mode2', 'neff')
ng = emeApi.getdata('mode1', 'ng')

print(f'Índice mode1: {n1}\nÍndice mode2: {n2}\nÍndice de grupo: {ng}')

Índice mode1: [[2.3531702+0.j]]
Índice mode2: [[1.7501308+0.j]]
Índice de grupo: [[4.3458796+0.j]]


# FSR

$$FSR(\lambda) = \frac{\lambda ^2}{n_g(\lambda)\Delta L}$$

FSR = 1 nm, 10 nm e 20 nm

In [9]:
fsr = np.array([1.0, 10.0, 20.0])

delta_L = wavelength**2 / (fsr*nm*ng)

display(Math(r'\Delta L:'))
print(f"{delta_L.real / um} µm")

<IPython.core.display.Math object>

[[552.8224937   55.28224937  27.64112468]] µm
